In [1]:
import carla
print(carla.__file__)
import random


C:\Users\14404\anaconda3\envs\tf\lib\site-packages\carla\__init__.py


In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*firetruck*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [3]:
# get the car's position on the map 
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471264, z=-0.001234), Rotation(pitch=-0.000034, yaw=0.159198, roll=0.000047))


In [4]:
# initial spawn point is the same - just 0.6m higher off the ground
print(start_point)

Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [5]:
# send vehicle off
vehicle.set_autopilot(True)


In [6]:
# get actual position from the car moving
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)


Transform(Location(x=-57.282505, y=22.889214, z=-0.003519), Rotation(pitch=0.030401, yaw=-15.440934, roll=0.208863))


In [7]:
# now look at the map
town_map = world.get_map()


In [8]:
type(town_map)


carla.libcarla.Map

In [9]:
print(town_map)


Map(name=Carla/Maps/Town10HD_Opt)


In [10]:
roads = town_map.get_topology()


In [11]:
print(roads)

[(<carla.libcarla.Waypoint object at 0x0000015C5F0C1B60>, <carla.libcarla.Waypoint object at 0x0000015C5F0C1BD0>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C1C40>, <carla.libcarla.Waypoint object at 0x0000015C5F0C1CB0>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C1D20>, <carla.libcarla.Waypoint object at 0x0000015C5F0C1D90>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C1E00>, <carla.libcarla.Waypoint object at 0x0000015C5F0C1E70>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C1EE0>, <carla.libcarla.Waypoint object at 0x0000015C5F0C1F50>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C1FC0>, <carla.libcarla.Waypoint object at 0x0000015C5F0C2030>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C20A0>, <carla.libcarla.Waypoint object at 0x0000015C5F0C2110>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C2180>, <carla.libcarla.Waypoint object at 0x0000015C5F0C21F0>), (<carla.libcarla.Waypoint object at 0x0000015C5F0C2260>, <carla.libcarla.Waypoint objec

In [12]:
#have a look at a waypoint - it is transform wrapped as a waypoint
print(roads[0][0])


Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [13]:
len(roads)


200

In [14]:
# making a route
# from one position to another
# import some code coming with the sim
import sys
sys.path.append('C:/Users/14404/CARLA_0.9.15/WindowsNoEditor/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner


In [15]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [16]:
# using the code to plan the route and then draw it in the simulator

sampling_resolution = 2
grp = GlobalRoutePlanner(town_map, sampling_resolution)
point_a = carla.Location(x=-64.644844, y=24.471010, z=0.600000)
point_b = carla.Location(x=109.929878, y=-9.334196, z=0.000000)

route = grp.trace_route(point_a, point_b)

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
    color = carla.Color(r = 0, g = 0, b = 255), life_time = 120,
    persistent_lines = True)

In [17]:
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

# start first car in already defined start point
truck = world.try_spawn_actor(truck_bp[0],start_point)


In [18]:
# tweak spectator position to watch the show

spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = start_point.rotation.yaw -155))

spectator.set_transform(spectator_pos)


In [19]:
# drop the Mini the sky - watch what happens after

#spawn it first somewhere else
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10])

mini_pos = carla.Transform(start_point.location + carla.Location(x=-4,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)

In [20]:
import time
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

#start first car in alredy defined start point
truck = world.try_spawn_actor(truck_bp[0], start_point)
time.sleep(3)

vw_bp = world.get_blueprint_library().filter('*volkswagen*')

vw = world.try_spawn_actor(vw_bp[0], spawn_points[20])
vw_pos = carla.Transform(start_point.location + carla.Location(x=1,z=6),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
vw.set_transform(vw_pos)
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10])

mini_pos = carla.Transform(start_point.location + carla.Location(x=1,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)

In [21]:
vw_bp = world.get_blueprint_library().filter('*volkswagen*')

In [22]:
vehicle_all = world.get_blueprint_library()
print(vehicle_all)

[ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra]), ActorBlueprint(id=vehicle.audi.a2,tags=[vehicle, audi, a2]), ActorBlueprint(id=static.prop.plantpot04,tags=[static, prop, plantpot04]), ActorBlueprint(id=static.prop.bench02,tags=[bench02, static, prop]), ActorBlueprint(id=vehicle.mercedes.coupe_2020,tags=[vehicle, mercedes, coupe_2020]), ActorBlueprint(id=vehicle.audi.tt,tags=[vehicle, audi, tt]), ActorBlueprint(id=walker.pedestrian.0037,tags=[0037, walker, pedestrian]), ActorBlueprint(id=vehicle.harley-davidson.low_rider,tags=[harley-davidson, vehicle, low_rider]), ActorBlueprint(id=vehicle.bmw.grandtourer,tags=[vehicle, bmw, grandtourer]), ActorBlueprint(id=static.prop.trashcan05,tags=[static, prop, trashcan05]), ActorBlueprint(id=walker.pedestrian.0038,tags=[walker, pedestrian, 0038]), ActorBlueprint(id=vehicle.carlamotors.firetruck,tags=[vehicle, carlamotors, firetruck]), ActorBlueprint(id=vehicle.micro.microlino,tags=[vehicle, microlino, micro]), ActorBluepri